导入所需的库

In [15]:
import numpy as np
from sklearn import datasets, model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

mnist  = datasets.fetch_mldata('MNIST original')
data,target = mnist.data, mnist.target

查看数据集

In [16]:
data.shape,target.shape

((70000, 784), (70000,))

构建数据集

In [19]:
index = np.random.choice(len(target), 70000, replace=False)

def mk_dataset(size):
    train_img = [data[i] for i in index[:size]]
    train_img = np.array(train_img)
    train_target = [target[i] for i in index[:size]]
    train_target = np.array(train_target)
    return train_img, train_target

In [20]:
fifty_x, fifty_y = mk_dataset(50000)

查看构建的数据集大小

In [21]:
fifty_x.shape, fifty_y.shape

((50000, 784), (50000,))

In [22]:
twenty_x, twenty_y = mk_dataset(20000)
twenty_x.shape, twenty_y.shape

((20000, 784), (20000,))

构建一个大小为10000的测试集

In [25]:
test_img = [data[i] for i in index[60000:70000]]
test_img1 = np.array(test_img)
test_target = [target[i] for i in index[60000:70000]]
test_target1 = np.array(test_target)
test_img1.shape, test_target1.shape

((10000, 784), (10000,))

直接调用KNN模型，并不涉及模型的内在设计

In [26]:
def skl_knn(k, test_data, test_target, stored_data, stored_target):
    classifier = KNeighborsClassifier(n_neighbors=k)
    classifier.fit(stored_data, stored_target)
#     用stored_data，stored_target训练分类器
    y_pred = classifier.predict(test_data)
    print(classification_report(test_target, y_pred))    

In [ ]:
%%time
skl_knn(5, test_img1, test_target1, fifty_x, fifty_y)

In [ ]:
%%time
skl_knn(5, test_img1, test_target1, twenty_x, twenty_y)

使用余弦特征构建一个更快的分类器

In [ ]:
import heapq 
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classificatiom_report
from sklearn import datasets, model_selection

def cos_knn(k, test_data, test_target, stored_data, stored_target):
    cosim = cosine_similarity(test_data, stored_data)
    top = [(heapq.nlargest((k), range(len(i)), i.take)) for i in cosim]
    top = [[stored_target[j] for j in [i:k]] for i in top]
    pred = [max(set(i), key=i.count) for i in top]
    pred = np.array(pred)
    print(classification_report(test_target, pred))    